In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #3CB371;"><b style="color:black;">Principal Component Analysis (PCA).</b></h1></center>

![](https://image.slidesharecdn.com/pca-lda-ppt-190704130157/95/principal-component-analysis-pca-and-lda-ppt-slides-1-638.jpg?cb=1562246425)slideshare.net

In [ ]:
df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/5802.csv", low_memory=False)
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#Handle numerical features with nan value
numerical_nan = [feature for feature in df.columns if df[feature].isna().sum()>1 and df[feature].dtypes!='O']
numerical_nan

In [ ]:
df[numerical_nan].isna().sum()

In [ ]:
## Replacing the numerical Missing Values

for feature in numerical_nan:
    ## We will replace by using median since there are outliers
    median_value=df[feature].median()
    
    df[feature].fillna(median_value,inplace=True)
    
df[numerical_nan].isnull().sum()

#Run pd.to_datetime. That will Date "become" int64 instead of datetime.

https://stackoverflow.com/questions/62759022/typeerror-invalid-type-promotion-while-fitting-a-logistic-regression-model-in-s
Answered by Sal_H

In [ ]:
#https://stackoverflow.com/questions/62759022/typeerror-invalid-type-promotion-while-fitting-a-logistic-regression-model-in-s
#Answered by Sal_H

df['time'] = pd.to_datetime(df['time'],infer_datetime_format=True)

In [ ]:
#https://stackoverflow.com/questions/62759022/typeerror-invalid-type-promotion-while-fitting-a-logistic-regression-model-in-s
#Answered by Sal_H

df['time']=df['time'].apply(lambda x: x.toordinal())

In [ ]:
x=df.copy()
x.drop('engagement_index',axis=1,inplace=True)
y=df['engagement_index']

#Encoding Categorical Features

In [ ]:
cat=[i for i in x.columns if x[i].dtypes=='O']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
encode=OrdinalEncoder()
labe=LabelEncoder()

In [ ]:
#Code by Ayush Yadav https://www.kaggle.com/smokingkrils/indian-e-commerce-analysis-and-customer-retention/notebook

#using ordinal encoder for independent features
for i in cat:
    x[i]=encode.fit_transform(x[i].values.reshape(-1,1))

#Using label encoder for Label Column
y=labe.fit_transform(y)

#Scale the data to the range between 0 and 1 before using PCA!

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [ ]:
xd=scaler.fit_transform(x)
x=pd.DataFrame(xd,columns=x.columns)

#Using Feature importance of Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
m=RandomForestRegressor()
m.fit(x,y)

In [ ]:
#Code by Ayush Yadav https://www.kaggle.com/smokingkrils/indian-e-commerce-analysis-and-customer-retention/notebook

#plot graph of feature importances for better visualization
feat_importances = pd.Series(m.feature_importances_, index=x.columns)
plt.figure(figsize=(10,8))
feat_importances.nlargest(10).plot(kind='barh', color = 'green')
plt.show()

#Using chi2 test

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
selection = SelectKBest(score_func=chi2, k='all')
fit = selection.fit(x,y)

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']  #naming the dataframe columns

In [ ]:
print(featureScores.nlargest(10,'Score'))  #print10??? best features I have only 4!
feat=list(featureScores.nlargest(10,'Score')['Features'])

#PCA

In [ ]:
#Code by Ayush Yadav https://www.kaggle.com/smokingkrils/indian-e-commerce-analysis-and-customer-retention/notebook

from sklearn.decomposition import PCA
pca = PCA().fit(x)

#Attention xi=np.arange (1,4, step=1) I wrote 4 because there are 4 features.

In [ ]:
#Code by Ayush Yadav https://www.kaggle.com/smokingkrils/indian-e-commerce-analysis-and-customer-retention/notebook

fig, ax = plt.subplots(figsize=(20,10))
xi = np.arange(1, 4, step=1)
yi = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, yi, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 3, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=1, color='r', linestyle='-')
plt.text(0.5, 0.85, '100% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

On the plotted chart above, we see what number of principal components we need.
https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/

Number of components I wrote one less than the 4 features.

On the original code was 29 (features choosen).

#Don’t choose the number of components manually. (I did because I´m still learning)

Instead of that, use the option that allows you to set the variance of the input that is supposed to be explained by the generated components.


In [ ]:
#Code by Ayush Yadav https://www.kaggle.com/smokingkrils/indian-e-commerce-analysis-and-customer-retention/notebook

pca=PCA(n_components=3)
x=pca.fit_transform(x)
x=pd.DataFrame(x)
x.head()

In [ ]:
#Code by saiabhiteja Chepuri  https://www.kaggle.com/saiabhitejachepuri/advance-regression-05/notebook

#assiging X values
X=df.drop(["engagement_index",'lp_id'],axis=1)
X = X.values
X.shape

In [ ]:
y = df['engagement_index']
y = y.values
y.shape

#We don´t have test in this Dataset. Save the snippet for the next time.

I replaced dftest to df, since we don't have test.

In [ ]:
#assiging xtest values
xtest=df.drop(['lp_id'],axis=1)
xtest = xtest.values
xtest.shape

In [ ]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 69)

#Selecting Models

In [ ]:
from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor,XGBRFRegressor

#Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
pred = lr.predict(X_test)

In [ ]:
#Code by saiabhiteja Chepuri  https://www.kaggle.com/saiabhitejachepuri/advance-regression-05/notebook

#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid', rc = {'figure.figsize':(8,6)})
sns.distplot(pred-y_test,kde=False, color='red');

#Ridge Regressor

In [ ]:
rc = RidgeCV(cv = 20)
rc.fit(X_train,y_train)

In [ ]:
pred = rc.predict(X_test)

In [ ]:
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(8,6)})
sns.distplot(pred-y_test,kde=False);

#Lasso

In [ ]:
ls = LassoCV(cv = 20)
ls.fit(X_train,y_train)

In [ ]:
pred = ls.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False);

#Random Forest

In [ ]:
rf = RandomForestRegressor(max_depth=4,n_estimators = 500)
rf.fit(X_train,y_train)

In [ ]:
pred = rf.predict(X_test)

In [ ]:
pred = rf.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False, color='red');

#XGBoost

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train,y_train)

In [ ]:
pred = xgb.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False, color='red');

In [ ]:
xgb = XGBRFRegressor()
xgb.fit(X_train,y_train)

In [ ]:
pred = xgb.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False, color='green');

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gd = GradientBoostingRegressor(n_estimators=300,max_depth=6)
gd.fit(X_train,y_train)

In [ ]:
pred = gd.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(8,6)})
sns.distplot(pred-y_test,kde=False, color='green');

That's all for now with PCA, Scaling and Regressors